In [1]:
import numpy as np
import scipy
import pandas
import plotly.graph_objects as go
import plotly.express as px

random_seed = 3

When running a statistical test we are often concerned with level of significance ($\alpha$) and power ($\beta$). The level of significance determines the probability that we would have a Type I error from our test, i.e. a false positive. Whereas the power relates to the probability of a Type II error, i.e. a false negative. The probability of a Type II error is given by $1-\beta$. Many studies use the values of $\alpha = 0.05$ and $\beta = 0.8$. If we were to construct a study where there was no difference between two cases with the following values we would expect to find that in 5% of the experiments that the test produces a statistically significant difference. 

The distributions that are being used are standard normal distributions, meaning that they have a mean of 0 and a standard deviation of 1. When determining the initial conditions for the experiment, we will calculate the sample size needed to detect a difference of 0.5 unit between the distributions, i.e. the effect size for the experiment is one. Where $Z_{1-\alpha/2}$ is the value of the standard normal distribution representing the $1-\alpha/2$ percentile and $Z_{1-\beta}$ is the absolute value of the standard normal distribution at the $1-\beta$ percentile.   

$\Huge
\begin{align}
n_i =& 2\left(\frac{Z_{1-\alpha/2} + Z_{1-\beta}}{\text{Effect Size}}\right)^{2} \\
n_i =& 2\left(\frac{Z_{0.95} + Z_{0.2}}{\text{0.1}}\right)^{2} \\
n_i =& 2\left(\frac{1.96 + 0.842}{\text{0.1}}\right)^{2} \\
n_i = & 62.8
\end{align}
$

We can produce a study with these alpha and beta values by having 63 observations in each of the study groups, for a total sample of 126 observations.

In [0]:
np.random.seed(random_seed)
def plot_experiment(difference=0.0,sample_size = 63):
    df = pandas.DataFrame()
    df["control"] = [np.random.normal() for _ in range(sample_size)]
    df["experiment"] = [np.random.normal()+difference for _ in range(sample_size)]
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=df['control'], name='Control'))
    fig.add_trace(go.Histogram(x=df['experiment'], name='Experiment'))
    fig.update_layout(barmode='overlay', xaxis_title_text='Value', yaxis_title_text='Count' )
    fig.update_traces(opacity=0.7)
    fig.show()

In [77]:
plot_experiment()

The sameness of the distributions becomes more clear as we increase the sample size.

In [89]:
plot_experiment(sample_size=10000)

We know that there should be no statistically significant difference between the control and experiment samples because they are constructed from the same distribution. We can test this using the 

In [84]:
statistic, p_value = scipy.stats.ttest_ind(a = df['control'],
                                           b = df['experiment'],
                                           alternative = 'two-sided')
print(f'The p-value for this test is {p_value}')

The p-value for this test is 0.27585149072039394


### From this we can already start to see that the randomness from sampling can have an impact on the results of the statistical test. If we run the these experiments 100 times we should begin to see false positive results from the test, i.e. p-values that are less than 0.05.

In [45]:
def run_experiment(sample_size, confidence_level = 0.95, difference = 0.0):
    control = [np.random.normal() for _ in range(sample_size)]
    experiment = [np.random.normal() + difference for _ in range(sample_size)]
    output = scipy.stats.ttest_ind(a = experiment, b = control, alternative = 'two-sided')
    p_value = output[1]
    return p_value, output.confidence_interval(confidence_level)

In [68]:
np.random.seed(random_seed)
sample_size = 63
false_positive_results = []
p_values = {}
confidence_intervals = {}
for i in range(100):
    p_values[i], confidence_intervals[i] = run_experiment(sample_size)
    if p_values[i] < 0.05:
        false_positive_results.append(p_values[i]) 
    else:
        pass
    
print(f"There were a total of {len(false_positive_results)} false positives")
for p in false_positive_results:
    print(f"\t p = {p:.4f}")

There were a total of 5 false positives
	 p = 0.0360
	 p = 0.0250
	 p = 0.0274
	 p = 0.0254
	 p = 0.0141


### We can put this another way by comparing the confidence interval for the mean difference between these two samples. We drew the samples out of the same distribution, so we would expect to produce a mean difference of zero to be in the 95% confidence interval when we fail to reject the null hypothesis. Our false positives will show as cases where zero is not contained in the 95% confidence interval for the mean difference between the samples.

In [69]:
df = pandas.DataFrame()
df['means'] = [(high+low)/2 for low, high in confidence_intervals.values()]
df['err'] = abs(df['means'] - [low for low, high in confidence_intervals.values()]) # Confidence interval is symmetric
df['Truth'] = ['True Result' if low <= 0.0 and 0.0 < high else 'False Positive' for low,high in confidence_intervals.values()]
df['p-value'] = p_values.values()

In [70]:
fig = px.scatter(df, x=df.index, y='means', error_y='err', color='Truth',
                 hover_data=['p-value'],
                 title="95% confidence intervals around the difference in population means")
fig.update_layout(width=1200, height=400, xaxis_title='Experiment Number', yaxis_title='Difference in Means')
fig.show()

We designed this experiment so that it would have a power of 0.8, meaning that it should miss a true difference in 20% of the experiments. The effect size was set to be 0.5 when designing the samples for the experiments. So, if we were to run 100 experiments, we would expect that 80% would reject the null hypothesis and 20% would fail to reject the null hypothesis. We can work through these experiments below. 

In [78]:
plot_experiment(difference=0.5)

If we ramp up the sample size we can more clearly see that these are truly different distributions

In [88]:
plot_experiment(difference=0.5, sample_size=10000)

In [72]:
np.random.seed(random_seed)
sample_size = 63
false_negative_results = []
p_values = {}
confidence_intervals = {}
for i in range(100):
    p_values[i], confidence_intervals[i] = run_experiment(sample_size, difference=0.5)
    if p_values[i] < 0.05:
        pass
    else:
        false_negative_results.append(p_values[i]) 
    
print(f"There were a total of {len(false_negative_results)} false negative")
for p in false_negative_results:
    print(f"\t p = {p:.4f}")

There were a total of 18 false negative
	 p = 0.0991
	 p = 0.0737
	 p = 0.0521
	 p = 0.3056
	 p = 0.0534
	 p = 0.1314
	 p = 0.0505
	 p = 0.0575
	 p = 0.1700
	 p = 0.0763
	 p = 0.0835
	 p = 0.0936
	 p = 0.7148
	 p = 0.0631
	 p = 0.1070
	 p = 0.2225
	 p = 0.0590
	 p = 0.1918


In [73]:
df = pandas.DataFrame()
df['means'] = [(high+low)/2 for low, high in confidence_intervals.values()]
df['err'] = abs(df['means'] - [low for low, high in confidence_intervals.values()]) # Confidence interval is symmetric
df['Truth'] = ['False Negative' if low <= 0.0 and 0.0 < high else 'True Result' for low,high in confidence_intervals.values()]
df['p-value'] = p_values.values()

In [74]:
fig = px.scatter(df, x=df.index, y='means', error_y='err', color='Truth',
                 hover_data=['p-value'],
                 title="95% confidence intervals around the difference in population means")
fig.update_layout(width=1200, height=400, xaxis_title='Experiment Number', yaxis_title='Difference in Means')
fig.show()